### Pipeline for QnA with Memory

In [51]:
####Notes####
# chunked data length can be atmost 16 for azureopenai

In [52]:
# !pip3 install langchain
# !pip install openai
# !pip3 install openai chromadb
# !pip3 install tiktoken

In [53]:
# !pip3 install unstructured
# !pip3 install pdf2image
# !pip3 install pdfminer
# !pip3 install pdfminer.six
# !pip3 install pymupdf 
# !pip3 install lxml
# !pip3 install lxml

In [54]:
import os
import sys
import glob
import re
import importlib
import langchain
from langchain.document_loaders import WebBaseLoader, UnstructuredPDFLoader, PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.indexes import VectorstoreIndexCreator
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.llms import AzureOpenAI
from langchain.prompts import PromptTemplate
sys.path.append(os.path.join(os.getcwd(), '../scripts'))
import helpers as h
import constants as c
import scraper as sc
importlib.reload(h)
importlib.reload(c)
importlib.reload(sc)

<module 'scraper' from '/Users/rachitjoshi/Documents/Finbot/notebooks/../scripts/scraper.py'>

#### Environment

In [32]:
os.environ["OPENAI_API_KEY"] = "6cdb659e5a9d402e80c212fe8ea26483"

In [33]:
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2023-03-15-preview"
os.environ["OPENAI_API_BASE"] = "https://test-chatgpt-flomoney.openai.azure.com/"
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

#### Paths

In [34]:
pdfs_path = os.path.join(os.getcwd(), '../data/pdfs')
merged_pdfs_path = os.path.join(os.getcwd(), '../data/pdfs_processed/merged')

#### Data

In [35]:
investorcom_pdfs = [ _ for _ in glob.glob(os.path.join(pdfs_path, '*.pdf')) if 'investorcom' in _]

In [43]:
web_list = sc.scrape_site("https://www.scientificamerican.com/")
web_list

[[]]

In [56]:
web_list = sc.scrape_site("https://www.espncricinfo.com/")
web_list

['https://www.espncricinfo.com//series/icc-cricket-world-cup-warm-up-matches-2023-24-1393958/bangladesh-vs-england-warm-up-1393964/live-cricket-score',
 'https://www.espncricinfo.com//series/icc-cricket-world-cup-warm-up-matches-2023-24-1393958/bangladesh-vs-england-warm-up-1393964/live-cricket-score',
 'https://www.espncricinfo.com//series/icc-cricket-world-cup-warm-up-matches-2023-24-1393958/bangladesh-vs-england-warm-up-1393964/live-cricket-score',
 'https://www.espncricinfo.com//series/icc-cricket-world-cup-warm-up-matches-2023-24-1393958/bangladesh-vs-england-warm-up-1393964/live-cricket-score',
 'https://www.espncricinfo.com//series/icc-cricket-world-cup-warm-up-matches-2023-24-1393958/match-schedule-fixtures-and-results',
 'https://www.espncricinfo.com//series/icc-cricket-world-cup-warm-up-matches-2023-24-1393958',
 'https://www.espncricinfo.com//series/icc-cricket-world-cup-warm-up-matches-2023-24-1393958/new-zealand-vs-south-africa-warm-up-1393965/live-cricket-score',
 'https:

In [37]:
web_list = [
    'https://www.scientificamerican.com/article/pangaea-ultima-the-next-supercontinent-may-doom-mammals-to-far-future-extinction/',
    'https://www.scientificamerican.com/article/ho-oleilana-a-billion-light-year-wide-bubble-of-galaxies-astounds-astronomers/',
    'https://www.scientificamerican.com/article/dead-trees-and-a-mysterious-cosmic-explosion-reveal-bigger-quake-risk-for-seattle/',
    'https://www.scientificamerican.com/article/streetlights-are-mysteriously-turning-purple-heres-why/',
    'https://www.scientificamerican.com/article/to-move-fast-quantum-maze-solvers-must-forget-the-past/',
]

In [30]:
web_list

[[]]

##### - Load

In [10]:
# loaded_data = []
# for pdf in investorcom_pdfs :
#     loaded_data.extend(UnstructuredPDFLoader(file_path=pdf).load())        

In [11]:
loaded_data = []
for page in web_list :
    loaded_data.extend(WebBaseLoader(page).load())       

In [12]:
len(loaded_data)

5

In [13]:
len(loaded_data[0].page_content)

7358

In [14]:
# for pdf in pdf_list :
#     try :
#         pdf_data.extend(UnstructuredPDFLoader(file_path=pdf).load())        
#     except NameError:
#         pdf_loader = UnstructuredPDFLoader(file_path=pdf)
#         pdf_data = pdf_loader.load()

##### - Split

In [15]:
chunk_size = c.prompt_max//c.retrieval_kwargs['k']
chunk_overlap = 0

In [16]:
data_splitter = RecursiveCharacterTextSplitter(
    chunk_size = chunk_size, chunk_overlap = chunk_overlap
)

In [17]:
chunked_data = data_splitter.split_documents(loaded_data)

In [18]:
len(chunked_data)

8

In [19]:
len(chunked_data[0].page_content)

7357

##### - Store

In [20]:
chunks_max = 15

In [21]:
vectorstore_engine = 'Finbot-embedding-2'

In [22]:
embedding_model = OpenAIEmbeddings(deployment=vectorstore_engine)

In [23]:
try:
    vectorstore.delete_collection()
except (ValueError, NameError):
    pass
for _ in range(0, len(chunked_data), chunks_max):
    vectorstore = Chroma.from_documents(
        documents=chunked_data[_: _ + chunks_max], embedding=embedding_model
    )

In [24]:
len(vectorstore.get()['documents'])

8

In [25]:
vectorstore.get().keys()

dict_keys(['ids', 'embeddings', 'metadatas', 'documents'])

##### - Retrieve

In [26]:
question = "what are the pros and cons of tesla stock and nvidia stock?"
docs = vectorstore.similarity_search_with_relevance_scores(question, k=7)
len(docs)

7

In [27]:
len(docs[0][0].page_content)

77

In [28]:
# print(docs[1][0].page_content)

##### - Generate

In [29]:
llm_engine = 'finbot-gpt'
llm_model = 'text-davinci-002'
temperature = 0
search_type = 'mmr'
retrieval_kwargs = {'k': 5, 'lambda_mult': 0.5, 'fetch_k':10}
answer_max_tokens = 512
save_folder = '../data/pdfs/'
prompt_template_file = os.path.join(os.getcwd(), '../scripts/prompt_template.txt')
pdf_list = [
    _ for _ in glob.glob(os.path.join(os.getcwd(), save_folder, '*.pdf'))
]
web_list = []
langchain.debug=False

In [30]:
############

In [31]:
base_llm = AzureOpenAI(
    engine=llm_engine, 
    model_name=llm_model, 
    temperature=temperature, 
    max_tokens=answer_max_tokens
)

/Users/rachitjoshi/finbot/lib/python3.9/site-packages/langchain/utils/utils.py:155: UserWarning: WARNING! engine is not default parameter.
                engine was transferred to model_kwargs.
                Please confirm that engine is what you intended.
  warnings.warn(


In [32]:
langchain.debug=True

In [33]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chains import ConversationalRetrievalChain

In [34]:
condense_question_template = os.path.join(
    os.getcwd(), '..', 'scripts/condense_question_template.txt'
)
condense_question_input = c.condense_question_input

In [35]:
chat_prompt = PromptTemplate.from_file(
    condense_question_template,
    input_variables=condense_question_input
)

In [36]:
qna_prompt_template = os.path.join(os.getcwd(), '..', 'scripts/qna_prompt_template.txt')
qna_prompt_input = c.qna_prompt_input
prompt_role = c.prompt_role

In [37]:
qna_prompt = PromptTemplate.from_file(
    qna_prompt_template,
    input_variables=qna_prompt_input,
    partial_variables={'role':prompt_role}
)

In [38]:
history_tokens = 2000
memory = ConversationSummaryBufferMemory(
    llm=base_llm, 
    memory_key="chat_history", 
    return_messages=True, 
    max_token_limit=history_tokens
)

In [39]:
qna_chain = ConversationalRetrievalChain.from_llm(
    base_llm, 
    retriever=vectorstore.as_retriever(
        search_type=search_type, search_kwargs=retrieval_kwargs
    ),
    memory=memory,
    condense_question_prompt=chat_prompt,
    combine_docs_chain_kwargs=dict(prompt=qna_prompt)
)

In [54]:
question = "what others reasons can be behind streetlights turning purple"
results = qna_chain({"question": question})

[chain/start] [1:chain:ConversationalRetrievalChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:ConversationalRetrievalChain > 2:chain:LLMChain] Entering Chain run with input:
{
  "question": "what others reasons can be behind streetlights turning purple",
  "chat_history": "\nHuman: why is there a danger of earthquake in seattle\nAssistant:  Washington State’s Puget Sound could face previously unknown earthquake risks, according to a new study that has pinned down the date of an ancient earthquake using tree rings and the radiation left by a mysterious cosmic force. The study shows that multiple faults rupturing essentially simultaneously is a real possibility, which is currently not planned for in the area. “It’s showing that these complex, large earthquakes are a real possibility,” says Bryan Black, a dendrochronologist at the University of Arizona and lead author of the new study. The “one-two punch” of the individual but closely timed quakes is also worrying. “

[llm/end] [1:chain:ConversationalRetrievalChain > 2:chain:LLMChain > 3:llm:AzureOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "What are the other reasons behind streetlights turning purple?<|im_end|>",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        }
      }
    ]
  ],
  "llm_output": {
    "token_usage": {
      "completion_tokens": 12,
      "total_tokens": 845,
      "prompt_tokens": 833
    },
    "model_name": "text-davinci-002"
  },
  "run": null
}
[chain/end] [1:chain:ConversationalRetrievalChain > 2:chain:LLMChain] s] Exiting Chain run with output:
{
  "text": "What are the other reasons behind streetlights turning purple?<|im_end|>"
}


Number of requested results 10 is greater than number of elements in index 8, updating n_results = 8


[chain/start] [1:chain:ConversationalRetrievalChain > 5:chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [1:chain:ConversationalRetrievalChain > 5:chain:StuffDocumentsChain > 6:chain:LLMChain] Entering Chain run with input:
{
  "question": "What are the other reasons behind streetlights turning purple?<|im_end|>",
  "context": "Streetlights Are Mysteriously Turning Purple. Here's Why - Scientific American \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to main contentSubscribeLatest IssuesMenuScientific AmericanSign In|NewslettersSearchSubscribeCloseCOVIDHealthMind & BrainEnvironmentTechnologySpace & PhysicsVideoPodcastsOpinionSubscribeCurrent IssueSign InNewslettersShareLatestSave $10 on digital!SubscribeEngineeringStreetlights Are Mysteriously Turning Purple. Here’s WhyNewly purple streetlights might seem innocuous, but they could affect driver and pedestrian safetyBy Karen Kwon on September 27, 2023Share on FacebookShare on TwitterShare on RedditShare on Linke

[llm/end] [1:chain:ConversationalRetrievalChain > 5:chain:StuffDocumentsChain > 6:chain:LLMChain > 7:llm:AzureOpenAI] [3.06s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " The phosphor layer around the lights has been “delaminated”—peeled off—exposing the blue LED light underneath. Although blue LED lights are, in principle, deep blue in color, they do naturally have a bit of a purple and violet tint. It’s possible that the lamps themselves could be malfunctioning in various ways, but other malfunctions would produce different results.<|im_end|>",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        }
      }
    ]
  ],
  "llm_output": {
    "token_usage": {
      "completion_tokens": 78,
      "total_tokens": 8882,
      "prompt_tokens": 8804
    },
    "model_name": "text-davinci-002"
  },
  "run": null
}
[chain/end] [1:chain:ConversationalRetrievalChain > 5:chain:StuffDocumentsChain > 6:chain:LLMChain]

In [55]:
print(results['answer'])

 The phosphor layer around the lights has been “delaminated”—peeled off—exposing the blue LED light underneath. Although blue LED lights are, in principle, deep blue in color, they do naturally have a bit of a purple and violet tint. It’s possible that the lamps themselves could be malfunctioning in various ways, but other malfunctions would produce different results.<|im_end|>
